### Doc 360 Sample Activity

In [1]:
import polars as pl
import pandas as pd
import gc
from datetime import datetime, timedelta,date
from dateutil.relativedelta import relativedelta
import numpy as np
import json

In [2]:
# load variables from JSON
with open('vars_wk.json', 'r') as json_file:
    js = json.load(json_file)
roster_file = js['roster_file']
num_weeks_calls = js['num_weeks_calls']
bucket = js['bucket']
quarter_start = datetime.strptime(js['quarter_start'], '%Y-%m-%d').date()

dflib = f's3://{bucket}/BIT/dataframes/'
rst = f's3://{bucket}/BIT/roster/'

In [3]:
# Utility Functions -
def load(df, lib=dflib):
    globals()[df] = pl.read_parquet(f'{lib}{df}.parquet')

In [4]:
# Importing Dependencies
load('temp_samples')
load('roster')
load('MASTER_UNI')
roster2 = (
    pl.read_parquet(f'{rst}MasterRoster_{roster_file}.parquet')
    .with_columns(
        pl.col('SalesRepIID').cast(pl.Int64),
        Rep = pl.col('EmpFName')+' '+pl.col('EmpLName')
    ).select(['SalesRepIID','Rep'])
)

#fixes for vortex import -> Probably caused by Polars Upgrades
temp_samples = temp_samples.with_columns(pl.col('SalesRepIID').cast(pl.Int64))


In [5]:
iw_samples = (
    temp_samples
    .filter(pl.col('sample_week')<=num_weeks_calls)
    .filter(pl.col('CallDate')>= quarter_start)
    # .join(MASTER_UNI.select(['IID','Territory']),left_on = 'AttendeeIID', right_on = 'IID',how='left')
    # .join(roster, on = 'SalesRepIID' , how = 'left')
    # .filter(pl.col('Territory')==pl.col('GEO'))
)

In [6]:
# Processing =
def get_pid(desc):
    if desc == '290 mcg':
        return(5)
    elif desc =='145 mcg':
        return(4)
    elif desc =='72 mcg':
        return(3)
    return(0)

temp1 = (
    iw_samples
    .with_columns(pl.col('CallProductDescription').map_elements(get_pid,return_dtype=pl.Int64).alias('Product_id'))
    .select(['CallID','AttendeeIID','CallDate','SalesRepIID','CallProductQuantity','Product_id'])
    .join(roster2,on='SalesRepIID',how='left').drop('SalesRepIID')
    .group_by(['CallID','AttendeeIID','CallDate','Product_id','Rep'])
    .agg(Quantity=pl.col('CallProductQuantity').sum())
    .with_columns(
        pl.col('CallID').cast(pl.Int64),
        pl.lit(1).alias('Salesforce_id'),
    )
    .select('CallID','Product_id','AttendeeIID','Salesforce_id','CallDate','Rep','Quantity')
)
temp1.columns = ['SampleId','Product_id','DoctorId','Salesforce_id','Date','Rep','Quantity']

In [7]:
#Exporting Feeds-
OUT = 's3://vortex-staging-a65ced90/BIT/output/Doc360/'
temp1.to_pandas().to_csv(f'{OUT}Doc360_SampleActivity_Feed.txt', sep='|',lineterminator='\r\n',index=False)
print('Doc360_SampleActivity_Feed Exported !')

Doc360_SampleActivity_Feed Exported !
